In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from causadb import CausaDB
from causadb.plotting import plot_causal_graph, plot_causal_attributions
import os
import numpy as np

client = CausaDB(token=os.getenv("CAUSADB_TOKEN"))

In [3]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    "A": np.random.randn(100),
    "B": np.random.randn(100),
    "C": np.random.randn(100),
})

data.head()

,A,B,C
0,2.646655,0.061826,-0.951162
1,-0.996810,1.712514,0.755063
2,0.624841,-0.204726,0.736353
3,2.167268,2.168623,0.697739
4,-0.955820,-0.183882,-0.312336


In [14]:
df = pd.read_csv('lambda_data.csv')
df.head()

,lambda_name,workload,memory_size,billed_duration,memory_used,cost,cost_per_million
0,LGP,228083,256,5424.0,41.0,0.000023,22.60452
1,ONW,1345432,6144,9277.0,52.0,0.000928,927.88554
2,LZD,532240,1024,5046.0,44.0,0.000084,84.11682
3,GGN,588079,128,30000.0,41.0,0.000063,62.51250
4,EDU,1614122,8192,12516.0,54.0,0.001669,1669.13376


In [19]:
# Find null values
df.isnull().values.any()

False

In [22]:
client \
  .add_data("hnr-data") \
  .from_pandas(df[["workload", "memory_size", "billed_duration", "memory_used", "cost", "cost_per_million"]])

In [23]:
model = client.create_model("hnr-model")
model.set_nodes(["workload", "memory_size", "memory_used", "billed_duration", "cost_per_million"])
model.set_edges([
    ("workload", "memory_size"),
    ("workload", "memory_used"),
    ("memory_size", "memory_used"),
    ("memory_size", "billed_duration"),
    ("workload", "billed_duration"),
    ("memory_used", "billed_duration"),
    ("memory_size", "cost_per_million"),
    ("billed_duration", "cost_per_million")
])

model.train("hnr-data")

In [25]:
model.find_best_actions(
    targets={
        "cost_per_million": "minimise", 
        "billed_duration": "minimise"
    }, 
    actionable=["memory_size"],
    data=df[["workload", "memory_size", "billed_duration", "memory_used", "cost", "cost_per_million"]]
) # DF can't handle string entries, fix this! More importantly, only one recommendation is returned, but should be len(df)

{'workload': [228083, 1345432, 532240, 588079, 1614122, 1853655, 1186800, 743730, 1207414, 508983, 738874, 814321, 922159, 1824891, 1082907, 1441382, 1937804, 1483476, 1335200, 669204, 1489779, 1864052, 1702472, 1217033, 1448529, 1353397, 1162428, 1420373, 1764183, 1304940, 182984, 993764, 632915, 379363, 498746, 962829, 887715, 877922, 296964, 524230, 809065, 215438, 1875466, 1135125, 724582, 996802, 328834, 1144723, 292464, 1466405], 'memory_size': [256, 6144, 1024, 128, 8192, 6144, 6144, 1024, 2048, 128, 512, 1024, 1024, 6144, 512, 4096, 4096, 6144, 6144, 256, 4096, 6144, 8192, 8192, 4096, 6144, 1024, 1024, 6144, 4096, 256, 1024, 128, 128, 512, 512, 2048, 2048, 256, 256, 128, 128, 8192, 4096, 512, 1024, 128, 2048, 128, 6144], 'billed_duration': [5424.0, 9277.0, 5046.0, 30000.0, 12516.0, 14200.0, 7769.0, 7286.0, 6909.0, 30000.0, 14198.0, 7549.0, 9529.0, 14295.0, 22982.0, 10271.0, 13279.0, 10570.0, 9107.0, 23358.0, 8643.0, 14488.0, 11001.0, 8183.0, 8788.0, 9394.0, 13123.0, 19735.0, 13

,memory_size
0,1711.525458


In [13]:
model.find_best_actions(
    targets={"C": -1},
    actionable=["A"],
    data=data
)

{'A': [2.6466550175734453, -0.9968101568511808, 0.6248405776777354, 2.167267650839435, -0.955820113995187, -1.2834522835838675, -2.073303117306852, 2.2488477714464556, -0.022409828619735693, -1.8562507465012232, -0.04210598701197271, 1.3946949230279784, 2.1417077628164085, 0.3772409949974625, 0.28973841140303475, 1.2668893933883527, -0.09743663771281193, 0.40470284928892875, -0.31433784810185605, -0.23557380684340434, 0.7067634625510905, 1.0375639809912052, 0.3407037165715367, -0.6711362017757769, -0.7619561746080006, 0.2524384175311176, -0.2918868147941623, 1.037294705850295, 0.8461373740189365, -1.484330102879922, 1.276112384963727, 0.4025122470927633, 0.6919906336842802, 2.0133734053933567, -0.06457829775711059, -0.8636993879507239, 0.9693561267274167, -0.2837865977053893, -0.7540811091866615, -1.860485383657241, 1.619981271229795, -1.3194380220817243, 1.9504088398496386, -0.37187217112120385, -2.5272277468797624, 0.31053877252801165, -0.7990488092061173, 0.0404726658730539, 0.36892

,A
0,0.15052
